In [ ]:
import csv
import psycopg2

def clean_value(val, target_type='text'):
    """
    Cleans a single value for SQL insertion based on the expected target type.
    - Converts NA, null, blank to appropriate defaults
    - Supports text, int, float (numeric)
    """
    if val is None:
        val = ''
    val = val.strip()

    # Treat these as missing
    if val.lower() in {'', 'na', 'null'}:
        if target_type == 'int':
            return '0'
        elif target_type == 'float':
            return '0.0'
        else:  # default is text
            return ''
    
    # For actual values, enforce basic type casting (optional safety)
    if target_type == 'int':
        try:
            return str(int(float(val)))
        except:
            return '0'
    elif target_type == 'float':
        try:
            return str(float(val))
        except:
            return '0.0'
    else:
        return val


# Step 1: Preprocess
input_file = '../data/hw2/bus/stops.txt'
output_file = '../data/hw2/bus/stops_preprocessed.txt'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = [
        'stop_id', 'stop_code', 'stop_name', 'stop_desc', 'stop_lat', 'stop_lon',
        'zone_id', 'stop_url', 'location_type', 'parent_station', 'stop_timezone', 'wheelchair_boarding'
    ]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        writer.writerow({
            'stop_id': clean_value(row.get('stop_id')),
            'stop_code': clean_value(row.get('stop_code')),
            'stop_name': clean_value(row.get('stop_name')),
            'stop_desc': clean_value(row.get('stop_desc')),
            'stop_lat': clean_value(row.get('stop_lat'), target_type='float'),
            'stop_lon': clean_value(row.get('stop_lon'), target_type='float'),
            'zone_id': clean_value(row.get('zone_id')),
            'stop_url': clean_value(row.get('stop_url')),
            'location_type': clean_value(row.get('location_type'), target_type='int'),
            'parent_station': clean_value(row.get('parent_station')),
            'stop_timezone': clean_value(row.get('stop_timezone')),
            'wheelchair_boarding': clean_value(row.get('wheelchair_boarding'), target_type='int')
        })

print("Preprocessed file written to:", output_file)

# Step 2: Load into PostgreSQL
conn = psycopg2.connect(
    dbname='hw2',
    user='postgres',
    password='123456',
    host='localhost',
    port='5432'
)
conn.autocommit = True

with conn.cursor() as cur:
    cur.execute("CREATE SCHEMA IF NOT EXISTS septa;")

    cur.execute("""
        CREATE TABLE IF NOT EXISTS septa.bus_stops (
            stop_id TEXT,
            stop_code TEXT,
            stop_name TEXT,
            stop_desc TEXT,
            stop_lat DOUBLE PRECISION,
            stop_lon DOUBLE PRECISION,
            zone_id TEXT,
            stop_url TEXT,
            location_type INTEGER,
            parent_station TEXT,
            stop_timezone TEXT,
            wheelchair_boarding INTEGER
        );
    """)


with conn.cursor() as cur:
    with open(output_file, 'r', encoding='utf-8') as f:
        cur.copy_expert(
            "COPY septa.bus_stops FROM STDIN WITH CSV HEADER DELIMITER ','",
            f
        )
    print("Loaded preprocessed bus stops into septa.bus_stops")



Preprocessed file written to: ../data/hw2/bus/stops_preprocessed.txt
Loaded preprocessed bus stops into septa.bus_stops


In [14]:
# Preprocess and write cleaned file
input_file = '../data/hw2/bus/routes.txt'
output_file = '../data/hw2/bus/routes_preprocessed.txt'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = [
        'route_id', 'agency_id', 'route_short_name', 'route_long_name',
        'route_desc', 'route_type', 'route_url', 'route_color', 'route_text_color'
    ]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        writer.writerow({
            'route_id': clean_value(row.get('route_id')),
            'agency_id': clean_value(row.get('agency_id')),
            'route_short_name': clean_value(row.get('route_short_name')),
            'route_long_name': clean_value(row.get('route_long_name')),
            'route_desc': clean_value(row.get('route_desc')),
            'route_type': clean_value(row.get('route_type'), target_type='int'),
            'route_url': clean_value(row.get('route_url')),
            'route_color': clean_value(row.get('route_color')),
            'route_text_color': clean_value(row.get('route_text_color')),
        })

print("Preprocessed routes saved to:", output_file)

# Create table and load data
with conn.cursor() as cur:
    cur.execute("CREATE SCHEMA IF NOT EXISTS septa;")
    cur.execute("""
        CREATE TABLE IF NOT EXISTS septa.bus_routes (
            route_id TEXT,
            agency_id TEXT,
            route_short_name TEXT,
            route_long_name TEXT,
            route_desc TEXT,
            route_type TEXT,
            route_url TEXT,
            route_color TEXT,
            route_text_color TEXT
        );
    """)

    with open(output_file, 'r', encoding='utf-8') as f:
        cur.copy_expert(
            "COPY septa.bus_routes FROM STDIN WITH CSV HEADER DELIMITER ','",
            f
        )
    print("Loaded data into septa.bus_routes")

Preprocessed routes saved to: ../data/hw2/bus/routes_preprocessed.txt
Loaded data into septa.bus_routes


In [15]:
# Step 1: Preprocess trips.txt
input_file = '../data/hw2/bus/trips.txt'
output_file = '../data/hw2/bus/trips_preprocessed.txt'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = [
        'route_id', 'service_id', 'trip_id', 'trip_headsign',
        'trip_short_name', 'direction_id', 'block_id', 'shape_id',
        'wheelchair_accessible', 'bikes_allowed'
    ]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        writer.writerow({
            'route_id': clean_value(row.get('route_id')),
            'service_id': clean_value(row.get('service_id')),
            'trip_id': clean_value(row.get('trip_id')),
            'trip_headsign': clean_value(row.get('trip_headsign')),
            'trip_short_name': clean_value(row.get('trip_short_name')),
            'direction_id': clean_value(row.get('direction_id')),
            'block_id': clean_value(row.get('block_id')),
            'shape_id': clean_value(row.get('shape_id')),
            'wheelchair_accessible': clean_value(row.get('wheelchair_accessible'), target_type='int'),
            'bikes_allowed': clean_value(row.get('bikes_allowed'), target_type='int'),
        })

print("Preprocessed trips saved to:", output_file)

# Step 2: Create table and load into DB
with conn.cursor() as cur:
    cur.execute("CREATE SCHEMA IF NOT EXISTS septa;")
    cur.execute("""
        CREATE TABLE IF NOT EXISTS septa.bus_trips (
            route_id TEXT,
            service_id TEXT,
            trip_id TEXT,
            trip_headsign TEXT,
            trip_short_name TEXT,
            direction_id TEXT,
            block_id TEXT,
            shape_id TEXT,
            wheelchair_accessible INTEGER,
            bikes_allowed INTEGER
        );
    """)

    with open(output_file, 'r', encoding='utf-8') as f:
        cur.copy_expert(
            "COPY septa.bus_trips FROM STDIN WITH CSV HEADER DELIMITER ','",
            f
        )
    print("Loaded data into septa.bus_trips")


Preprocessed trips saved to: ../data/hw2/bus/trips_preprocessed.txt
Loaded data into septa.bus_trips


In [16]:
# Step 1: Preprocess shapes.txt
input_file = '../data/hw2/bus/shapes.txt'
output_file = '../data/hw2/bus/shapes_preprocessed.txt'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = [
        'shape_id',
        'shape_pt_lat',
        'shape_pt_lon',
        'shape_pt_sequence',
        'shape_dist_traveled'
    ]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        writer.writerow({
            'shape_id': clean_value(row.get('shape_id')),
            'shape_pt_lat': clean_value(row.get('shape_pt_lat'), target_type='float'),
            'shape_pt_lon': clean_value(row.get('shape_pt_lon'), target_type='float'),
            'shape_pt_sequence': clean_value(row.get('shape_pt_sequence'), target_type='int'),
            'shape_dist_traveled': clean_value(row.get('shape_dist_traveled'), target_type='float'),
        })

print("Preprocessed shapes saved to:", output_file)

# Step 2: Create table and load data
with conn.cursor() as cur:
    cur.execute("CREATE SCHEMA IF NOT EXISTS septa;")
    cur.execute("""
        CREATE TABLE IF NOT EXISTS septa.bus_shapes (
            shape_id TEXT,
            shape_pt_lat DOUBLE PRECISION,
            shape_pt_lon DOUBLE PRECISION,
            shape_pt_sequence INTEGER,
            shape_dist_traveled DOUBLE PRECISION
        );
    """)

    with open(output_file, 'r', encoding='utf-8') as f:
        cur.copy_expert(
            "COPY septa.bus_shapes FROM STDIN WITH CSV HEADER DELIMITER ','",
            f
        )
    print("Loaded data into septa.bus_shapes")


Preprocessed shapes saved to: ../data/hw2/bus/shapes_preprocessed.txt
Loaded data into septa.bus_shapes


In [17]:
# Step 1: Preprocess rail/stops.txt
input_file = '../data/hw2/rail/stops.txt'
output_file = '../data/hw2/rail/stops_preprocessed.txt'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = [
        'stop_id',
        'stop_name',
        'stop_desc',
        'stop_lat',
        'stop_lon',
        'zone_id',
        'stop_url'
    ]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        writer.writerow({
            'stop_id': clean_value(row.get('stop_id')),
            'stop_name': clean_value(row.get('stop_name')),
            'stop_desc': clean_value(row.get('stop_desc')),
            'stop_lat': clean_value(row.get('stop_lat'), target_type='float'),
            'stop_lon': clean_value(row.get('stop_lon'), target_type='float'),
            'zone_id': clean_value(row.get('zone_id')),
            'stop_url': clean_value(row.get('stop_url')),
        })

print("Preprocessed rail stops saved to:", output_file)

# Step 2: Create table and load data
with conn.cursor() as cur:
    cur.execute("CREATE SCHEMA IF NOT EXISTS septa;")
    cur.execute("""
        CREATE TABLE IF NOT EXISTS septa.rail_stops (
            stop_id TEXT,
            stop_name TEXT,
            stop_desc TEXT,
            stop_lat DOUBLE PRECISION,
            stop_lon DOUBLE PRECISION,
            zone_id TEXT,
            stop_url TEXT
        );
    """)

    with open(output_file, 'r', encoding='utf-8') as f:
        cur.copy_expert(
            "COPY septa.rail_stops FROM STDIN WITH CSV HEADER DELIMITER ','",
            f
        )
    print("Loaded data into septa.rail_stops")


Preprocessed rail stops saved to: ../data/hw2/rail/stops_preprocessed.txt
Loaded data into septa.rail_stops


In [ ]:
import geopandas as gpd
from sqlalchemy import create_engine, text
from geoalchemy2 import Geography
import os

# Load original GeoJSON and reproject from EPSG:2272 to EPSG:4326
gdf = gpd.read_file("../data/hw2/PWD_PARCELS.geojson", engine="fiona")
gdf = gdf.to_crs(epsg=4326)

# Rename geometry column to 'geog' and lowercase all field names
gdf = gdf.set_geometry("geometry").rename_geometry("geog")
gdf.columns = [col.lower() for col in gdf.columns]

# Build connection string
DB = {
    "user":     os.getenv("DB_USER",     "postgres"),
    "password": os.getenv("DB_PASSWORD", "123456"),
    "host":     os.getenv("DB_HOST",     "localhost"),
    "port":     os.getenv("DB_PORT",     "5432"),
    "database": os.getenv("DB_NAME",     "hw2"),
}
conn_str = (
    f"postgresql+psycopg2://"
    f"{DB['user']}:{DB['password']}@"
    f"{DB['host']}:{DB['port']}/{DB['database']}"
)
engine = create_engine(conn_str)

# Ensure PostGIS extension and schema
with engine.begin() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS postgis;"))
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS phl;"))

# Upload to PostGIS using GEOGRAPHY(MULTIPOLYGON, 4326)
gdf.to_postgis(
    name="pwd_parcels",
    con=engine,
    schema="phl",
    if_exists="replace",
    index=False,
    dtype={"geog": Geography("MULTIPOLYGON", srid=4326)}
)

print("phl.pwd_parcels successfully loaded with GEOGRAPHY column 'geog'")


phl.pwd_parcels successfully loaded with GEOGRAPHY column 'geog'


In [ ]:
import geopandas as gpd
from sqlalchemy import create_engine, text
from geoalchemy2 import Geography
import os

# Step 1: Read GeoJSON
gdf = gpd.read_file("../data/hw2/philadelphia-neighborhoods.geojson", engine="fiona")

# Step 2: Reproject to EPSG:4326 (optional safety — most GeoJSON already is)
gdf = gdf.to_crs(epsg=4326)

# Step 3: Rename geometry column + lowercase field names
gdf = gdf.set_geometry("geometry").rename_geometry("geog")
gdf.columns = [col.lower() for col in gdf.columns]

# Step 4: Connect to database
DB = {
    "user":     os.getenv("DB_USER",     "postgres"),
    "password": os.getenv("DB_PASSWORD", "123456"),
    "host":     os.getenv("DB_HOST",     "localhost"),
    "port":     os.getenv("DB_PORT",     "5432"),
    "database": os.getenv("DB_NAME",     "hw2"),
}
conn_str = (
    f"postgresql+psycopg2://"
    f"{DB['user']}:{DB['password']}@"
    f"{DB['host']}:{DB['port']}/{DB['database']}"
)
engine = create_engine(conn_str)

# Step 5: Ensure PostGIS + schema
with engine.begin() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS postgis;"))
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS phl;"))

# Step 6: Upload to PostGIS with GEOGRAPHY
gdf.to_postgis(
    name="neighborhoods",
    con=engine,
    schema="phl",
    if_exists="replace",
    index=False,
    dtype={"geog": Geography("MULTIPOLYGON", srid=4326)}
)

print("phl.neighborhoods successfully loaded with geography column 'geog'")


phl.neighborhoods successfully loaded with geography column 'geog'


In [ ]:
import geopandas as gpd
from sqlalchemy import create_engine, text
from geoalchemy2 import Geography
import os

# Step 1: Read the Census shapefile using Fiona
gdf = gpd.read_file("../data/hw2/tl_2020_42_bg.shp", engine="fiona")

# Step 2: Reproject from EPSG:4269 (NAD83) to EPSG:4326 (WGS 84)
gdf = gdf.to_crs(epsg=4326)

# Step 3: Standardize geometry column + lowercase all fields
gdf = gdf.set_geometry("geometry").rename_geometry("geog")
gdf.columns = [col.lower() for col in gdf.columns]

# Step 4: Create database connection string from env vars (or default fallback)
DB = {
    "user":     os.getenv("DB_USER",     "postgres"),
    "password": os.getenv("DB_PASSWORD", "123456"),
    "host":     os.getenv("DB_HOST",     "localhost"),
    "port":     os.getenv("DB_PORT",     "5432"),
    "database": os.getenv("DB_NAME",     "hw2"),
}
conn_str = (
    f"postgresql+psycopg2://"
    f"{DB['user']}:{DB['password']}@"
    f"{DB['host']}:{DB['port']}/{DB['database']}"
)
engine = create_engine(conn_str)

# Step 5: Ensure PostGIS and schema exist
with engine.begin() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS postgis;"))
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS census;"))

# Step 6: Upload to PostGIS using GEOGRAPHY(MULTIPOLYGON, 4326)
gdf.to_postgis(
    name="blockgroups_2020",
    con=engine,
    schema="census",
    if_exists="replace",
    index=False,
    dtype={"geog": Geography("MULTIPOLYGON", srid=4326)}
)

print("census.blockgroups_2020 successfully loaded with geography column 'geog'")


census.blockgroups_2020 successfully loaded with geography column 'geog'


In [ ]:
import pandas as pd
import psycopg2
import csv

# Step 1: Load the full CSV and preview columns
raw_df = pd.read_csv("../data/hw2/DECENNIALPL2020.P1-Data.csv")

# Step 2: Extract relevant columns
# Usually, GEO_ID is full geoid; NAME is the geography name; P1_001N is total pop.
df = pd.DataFrame()
df['geoid'] = raw_df['GEO_ID'].str.extract(r'US(\d+)$')  # Strip "US" prefix
df['geoname'] = raw_df['NAME']
df['total'] = raw_df['P1_001N']  # This is total population

# Optional sanity check
df = df.dropna(subset=['geoid', 'geoname', 'total'])

# Step 3: Save to temporary clean CSV for COPY
output_file = "../data/hw2/population_2020_clean.csv"
df.to_csv(output_file, index=False, quoting=csv.QUOTE_NONNUMERIC)

conn = psycopg2.connect(
    dbname="hw2",
    user="postgres",
    password="123456",
    host="localhost",
    port="5432"
)
conn.autocommit = True

# Step 4: Load into PostgreSQL
with conn.cursor() as cur:
    cur.execute("CREATE SCHEMA IF NOT EXISTS census;")
    cur.execute("""
        CREATE TABLE IF NOT EXISTS census.population_2020 (
            geoid TEXT,
            geoname TEXT,
            total INTEGER
        );
    """)

    with open(output_file, 'r', encoding='utf-8') as f:
        cur.copy_expert(
            "COPY census.population_2020 FROM STDIN WITH CSV HEADER",
            f
        )
    print("Loaded population data into census.population_2020")


Loaded population data into census.population_2020


C:\Users\fang_\AppData\Local\Temp\ipykernel_16584\53240181.py:6: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv("../data/hw2/DECENNIALPL2020.P1-Data.csv")
